In [1]:
%pip install langchain
%pip install langchain-openai
%pip install langchainhub
%pip install pypdf
%pip install chromadb

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import os

def _set_env_from_file(var: str, file_path: str = "openai_key.txt"):
    """
    Reads an API key from a specified file and sets it as an environment variable.
    """
    if not os.environ.get(var):
        try:
            # The 'with open' statement ensures the file is closed automatically
            with open(file_path, 'r') as f:
                # Read the first line and strip any leading/trailing whitespace
                key = f.readline().strip()

            if key:
                os.environ[var] = key
                print(f"Successfully loaded {var} from {file_path}")
            else:
                print(f"Warning: {file_path} is empty.")

        except FileNotFoundError:
            print(f"Error: Key file not found at {file_path}. Please create the file.")

# --- Execution ---
# Set the environment variable OPENAI_API_KEY from the file
_set_env_from_file('OPENAI_API_KEY')

Let's set up a study workflow using Jupyter Notebooks, LLMs, and langchain.

In [11]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [18]:
MODEL="gpt-4o-mini"

In [13]:
pdf_path = "C:\\Users\\gyanr\\Downloads\\pdf_langchain_test.pdf"
loader = PyPDFLoader(pdf_path) # LOAD
pdf_docs = loader.load_and_split() # SPLIT
pdf_docs

[Document(metadata={'producer': 'Pdftools SDK', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2025-10-05T12:35:19+00:00', 'source': 'C:\\Users\\gyanr\\Downloads\\pdf_langchain_test.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}, page_content='that suggested above. This short paper overviews some of the recent work on boosting, focusing\nespecially on the AdaBoost algorithm which has undergone intense theoretical study and empirical\ntesting. After introducing AdaBoost, we describe some of the basic underlying theory of boosting,\nincluding an explanation of why it often tends not to overﬁt. We also describe some experiments\nand applications using boosting.\nBackground\nBoosting has its roots in a theoretical framework for studying machine learning called the “PAC”\nlearning model, due to Valiant [46]; see Kearns and Vazirani [32] for a good introduction to this\nmodel. Kearns and Valiant [30, 31] were the ﬁrst to pose the question of whether a “weak” learn-\ning algorithm

In [14]:
doc_obj = pdf_docs[0]
doc_obj

Document(metadata={'producer': 'Pdftools SDK', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2025-10-05T12:35:19+00:00', 'source': 'C:\\Users\\gyanr\\Downloads\\pdf_langchain_test.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}, page_content='that suggested above. This short paper overviews some of the recent work on boosting, focusing\nespecially on the AdaBoost algorithm which has undergone intense theoretical study and empirical\ntesting. After introducing AdaBoost, we describe some of the basic underlying theory of boosting,\nincluding an explanation of why it often tends not to overﬁt. We also describe some experiments\nand applications using boosting.\nBackground\nBoosting has its roots in a theoretical framework for studying machine learning called the “PAC”\nlearning model, due to Valiant [46]; see Kearns and Vazirani [32] for a good introduction to this\nmodel. Kearns and Valiant [30, 31] were the ﬁrst to pose the question of whether a “weak” learn-\ning algorithm 

In [15]:
from IPython.display import display, Markdown

Markdown(doc_obj.page_content)

that suggested above. This short paper overviews some of the recent work on boosting, focusing
especially on the AdaBoost algorithm which has undergone intense theoretical study and empirical
testing. After introducing AdaBoost, we describe some of the basic underlying theory of boosting,
including an explanation of why it often tends not to overﬁt. We also describe some experiments
and applications using boosting.
Background
Boosting has its roots in a theoretical framework for studying machine learning called the “PAC”
learning model, due to Valiant [46]; see Kearns and Vazirani [32] for a good introduction to this
model. Kearns and Valiant [30, 31] were the ﬁrst to pose the question of whether a “weak” learn-
ing algorithm which performs just slightly better than random guessing in the PAC model can be
“boosted” into an arbitrarily accurate “strong” learning algorithm. Schapire [38] came up with the
ﬁrst provable polynomial-time boosting algorithm in 1989. A year later, Freund [17] developed
a much more efﬁcient boosting algorithm which, although optimal in a certain sense, neverthe-
less suffered from certain practical drawbacks. The ﬁrst experiments with these early boosting
algorithms were carried out by Drucker, Schapire and Simard [16] on an OCR task.
AdaBoost
The AdaBoost algorithm, introduced in 1995 by Freund and Schapire [23], solved many of the
practical difﬁculties of the earlier boosting algorithms, and is the focus of this paper. Pseudocode
for AdaBoost is given in Fig. 1. The algorithm takes as input a training set  	
 
where each  belongs to some domain or instance space  , and each label 	 is in some label
set  . For most of this paper, we assume    !"#%$ ; later, we discuss extensions to the
multiclass case. AdaBoost calls a given weak or base learning algorithm repeatedly in a series of
rounds &'(!*) . One of the main ideas of the algorithm is to maintain a distribution or set of
weights over the training set. The weight of this distribution on training example + on round & is
denoted ,.-/ +0
 . Initially, all weights are set equally, but on each round, the weights of incorrectly
classiﬁed examples are increased so that the weak learner is forced to focus on the hard examples
in the training set.
The weak learner’s job is to ﬁnd a weak hypothesis 12-435 6 7%"#%$ appropriate for the
distribution ,8- . The goodness of a weak hypothesis is measured by its error
9
-:<;>=@?A2BCD1E- 
GF HJIK L
JM NBPORQSUTWV
XZY
S
,#- +0

Notice that the error is measured with respect to the distribution ,[- on which the weak learner
was trained. In practice, the weak learner may be an algorithm that can use the weights ,\- on the
training examples. Alternatively, when this is not possible, a subset of the training examples can
be sampled according to ,8- , and these (unweighted) resampled examples can be used to train the
weak learner.
Relating back to the horse-racing example, the instances : correspond to descriptions of horse
races (such as which horses are running, what are the odds, the track records of each horse, etc.)
2

In [19]:
embeddings = OpenAIEmbeddings() # EMBED
embeddings
vectordb = Chroma.from_documents(pdf_docs, embedding=embeddings) # STORE


# Definition of a [retriever](https://python.langchain.com/docs/modules/data_connection/retrievers/#:~:text=A%20retriever%20is,Document's%20as%20output.):

# > A retriever is an interface that returns documents given an unstructured query. It is more general than a vector store. A retriever does not need to be able to store documents, only to return (or retrieve) them. Vector stores can be used as the backbone of a retriever, but there are other types of retrievers as well.
retriever = vectordb.as_retriever() 
# retriever
llm = ChatOpenAI(model=MODEL, temperature=0)
# source: https://python.langchain.com/v0.2/docs/tutorials/pdf_qa/#question-answering-with-rag

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# prompt
question_answer_chain = create_stuff_documents_chain(llm, prompt)

In [20]:
# question_answer_chain
# This method `create_stuff_documents_chain` [outputs an LCEL runnable](https://arc.net/l/quote/bnsztwth)
query = "What are the key components of the transformer architecture?"
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

results = rag_chain.invoke({"input": query})

results

{'input': 'What are the key components of the transformer architecture?',
 'context': [Document(metadata={'moddate': '2025-10-05T12:35:19+00:00', 'creator': 'PyPDF', 'source': 'C:\\Users\\gyanr\\Downloads\\pdf_langchain_test.pdf', 'creationdate': '', 'page_label': '3', 'producer': 'Pdftools SDK', 'total_pages': 4, 'page': 2}, page_content='in the margin values. This seems to be especially so when there are only a few relevant\nvariables so that \x07 can be very sparse. For instance, suppose the weak hypotheses all have\nrange \x1c\x1e\x1d \x1f!\x05\x0c"#\x1f%$ and that the label \x07 on all examples can be computed by a majority vote of\n\x00 \nof the weak hypotheses. In this case, it can be shown that if the number of relevant weak\nhypotheses\n\x00 \nis a small fraction of the total number of weak hypotheses then the margin\nassociated with AdaBoost will be much larger than the one associated with support vector\nmachines.\n\x05\nThe computation requirements are different. The comput

In [21]:
from IPython.display import Markdown

final_answer = results["answer"]

Markdown(final_answer)

The key components of the transformer architecture include the encoder and decoder layers, which consist of multi-head self-attention mechanisms and feed-forward neural networks. Additionally, transformers utilize positional encoding to retain the order of input sequences, and layer normalization is applied to stabilize training. The architecture is designed to process sequences in parallel, making it efficient for tasks like natural language processing.

In [22]:
query_summary = "Write a simple bullet points summary about this paper"

 # adding chat history so the model remembers previous questions
output = rag_chain.invoke({"input": query_summary})

Markdown(output["answer"])

- The paper focuses on the AdaBoost algorithm, a prominent boosting method in machine learning.
- It discusses the theoretical foundations of boosting, particularly the PAC learning model, and the evolution of boosting algorithms.
- AdaBoost improves upon earlier algorithms by effectively managing weights on training examples, enhancing performance on difficult cases.
- The algorithm is fast, simple, and requires no prior knowledge about the weak learner, making it versatile for various applications.
- Empirical tests show AdaBoost's effectiveness in tasks like OCR, text categorization, and natural language processing, often outperforming other methods.

In [23]:
def ask_pdf(pdf_qa,query):
    print("QUERY: ",query)
    result = pdf_qa.invoke({"input": query})
    answer = result["answer"]
    print("ANSWER", answer)
    return answer


ask_pdf(rag_chain,"How does the self-attention mechanism in transformers differ from traditional sequence alignment methods?")

QUERY:  How does the self-attention mechanism in transformers differ from traditional sequence alignment methods?
ANSWER The self-attention mechanism in transformers allows each element in a sequence to attend to all other elements, enabling the model to capture long-range dependencies and contextual relationships effectively. In contrast, traditional sequence alignment methods typically focus on pairwise comparisons, aligning sequences based on local similarities without considering the broader context of the entire sequence. This results in transformers being more flexible and powerful for tasks involving complex relationships in sequential data.


'The self-attention mechanism in transformers allows each element in a sequence to attend to all other elements, enabling the model to capture long-range dependencies and contextual relationships effectively. In contrast, traditional sequence alignment methods typically focus on pairwise comparisons, aligning sequences based on local similarities without considering the broader context of the entire sequence. This results in transformers being more flexible and powerful for tasks involving complex relationships in sequential data.'

In [24]:
quiz_questions = ask_pdf(rag_chain, "Quiz me with 3 simple questions on the positional encodings and the role they play in transformers.")

quiz_questions

QUERY:  Quiz me with 3 simple questions on the positional encodings and the role they play in transformers.
ANSWER 1. What is the primary purpose of positional encodings in transformer models?  
2. How do positional encodings help transformers understand the order of input sequences?  
3. What are two common methods for implementing positional encodings in transformers?


'1. What is the primary purpose of positional encodings in transformer models?  \n2. How do positional encodings help transformers understand the order of input sequences?  \n3. What are two common methods for implementing positional encodings in transformers?'

In [25]:
llm = ChatOpenAI(model=MODEL, temperature=0.0)

In [26]:
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List

class QA(BaseModel):
    questions: List[str] = Field(description='List of questions about a given context.')

C:\Users\gyanr\gyan-python-workspace\jup-workspace\venv\Lib\site-packages\IPython\core\interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [27]:
from langchain_core.prompts.chat import SystemMessagePromptTemplate, HumanMessagePromptTemplate

template = f"You transform unstructured questions about a topic into a structured list of questions."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_message_prompt = HumanMessagePromptTemplate.from_template("Return ONLY a PYTHON list containing the questions in this text: {questions}")

In [28]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt,human_message_prompt])

In [29]:
quiz_chain = chat_prompt | llm.with_structured_output(QA)

C:\Users\gyanr\gyan-python-workspace\jup-workspace\venv\Lib\site-packages\langchain_openai\chat_models\base.py:1900: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


In [30]:
questions_list = quiz_chain.invoke({"questions": quiz_questions})
questions_list

QA(questions=['What is the primary purpose of positional encodings in transformer models?', 'How do positional encodings help transformers understand the order of input sequences?', 'What are two common methods for implementing positional encodings in transformers?'])

In [31]:
questions = questions_list.questions

In [32]:
questions

['What is the primary purpose of positional encodings in transformer models?',
 'How do positional encodings help transformers understand the order of input sequences?',
 'What are two common methods for implementing positional encodings in transformers?']

In [33]:
# the questions variable was created within the string inside the `questions_list` variable.
answers = []
for q in questions:
    answers.append(ask_pdf(rag_chain,q))

QUERY:  What is the primary purpose of positional encodings in transformer models?
ANSWER The primary purpose of positional encodings in transformer models is to provide information about the order of the input tokens, as transformers do not have a built-in mechanism to capture sequence order due to their parallel processing nature. Positional encodings allow the model to differentiate between tokens based on their positions in the sequence, enabling it to understand the relationships and context of the tokens effectively. This is crucial for tasks involving sequential data, such as natural language processing.
QUERY:  How do positional encodings help transformers understand the order of input sequences?
ANSWER Positional encodings provide information about the position of each token in the input sequence, allowing transformers to capture the order of the tokens since they lack inherent sequential processing. By adding these encodings to the input embeddings, transformers can different

In [34]:
evaluations = []

for q,a in zip(questions, answers):
    # Check for results
    evaluations.append(ask_pdf(rag_chain,f"Is this: {a} the correct answer to this question: {q} according to the paper? Return ONLY '''YES''' or '''NO'''. Output:"))

evaluations

QUERY:  Is this: The primary purpose of positional encodings in transformer models is to provide information about the order of the input tokens, as transformers do not have a built-in mechanism to capture sequence order due to their parallel processing nature. Positional encodings allow the model to differentiate between tokens based on their positions in the sequence, enabling it to understand the relationships and context of the tokens effectively. This is crucial for tasks involving sequential data, such as natural language processing. the correct answer to this question: What is the primary purpose of positional encodings in transformer models? according to the paper? Return ONLY '''YES''' or '''NO'''. Output:
ANSWER YES
QUERY:  Is this: Positional encodings provide information about the position of each token in the input sequence, allowing transformers to capture the order of the tokens since they lack inherent sequential processing. By adding these encodings to the input embedd

['YES', 'YES', 'NO']

In [35]:
scores = []

yes_count = evaluations.count('YES')
score = str(yes_count/len(evaluations) * 100) + "%"
print(score)

66.66666666666666%


A more 'langchain way' to do this would be:

In [36]:
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

prompt_eval_template = """
You take in context, a question and a generated answer and you output ONLY a score of YES if the answer is correct,
or NO if the answer is not correct.

<context>
{context}
<context>

<question>
{question}
<question>

<answer>
{answer}
<answer>
"""

prompt_eval = ChatPromptTemplate.from_template(prompt_eval_template)

answer_eval_chain = (
    {
        'context': lambda x: format_docs(x['context']),
        'question': lambda x: x['question'],
        'answer': lambda x: x['answer']
        }
    ) | prompt_eval | llm | StrOutputParser()

In [37]:
evaluations = []
for q,a in zip(questions, answers):
    evaluations.append(answer_eval_chain.invoke({'context': pdf_docs, 'question': q, 'answer': a}))

In [38]:
evaluations

['YES', 'YES', 'YES']

In [39]:
scores = []

yes_count = evaluations.count('YES')
score = str(yes_count/len(evaluations) * 100) + "%"
print(score)

100.0%


In this example notebook we introduced a few interesting ideas:
1. Structured outputs
2. Some simple evaluation of rag answers using the 'llm-as-a-judge' strategy